In [28]:
# imports
import requests
import time
import pandas as pd

In [29]:
# --- Global Configuration ---
API_KEY = "RGAPI-ca35851d-28f7-480f-a51c-2b5798f68e06"  # Replace with your Riot API key
REGION = "europe"           # Replace with your region (e.g., na1, eun1, etc.)
MATCH_REGION = "europe"   # Match region (e.g., americas, asia, europe)
BASE_URL = f"https://{REGION}.api.riotgames.com"
MATCH_URL = f"https://{MATCH_REGION}.api.riotgames.com"
#nay 1401 puuid = "IvsWwkcWhgfifuaSvv9XoWw8OtzFDFvHj0lZpyN6yO-jSROwuQRiH2KczwahTZezGnRIGLUmfOJyug"

In [30]:
# helper functions
#request url : https://europe.api.riotgames.com/riot/account/v1/accounts/by-riot-id/nay/1401?api_key=RGAPI-ca35851d-28f7-480f-a51c-2b5798f68e06

In [38]:
def get_summoner_info(summoner_name, summoner_tagline):
    """Fetch summoner info by name and tagline."""
    # Include the API key as a query parameter
    url = f"{BASE_URL}/riot/account/v1/accounts/by-riot-id/{summoner_name}/{summoner_tagline}?api_key={API_KEY}"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching summoner info: {response.status_code}, {response.text}")
        return None


In [32]:
def get_match_ids(puuid, count=20):
    """Fetch match IDs for a given summoner."""
    url = f"{MATCH_URL}/lol/match/v5/matches/by-puuid/{puuid}/ids?count={count}"
    headers = {"X-Riot-Token": API_KEY}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching match IDs: {response.status_code}, {response.text}")
        return []

In [33]:
def get_match_details(match_id):
    """Fetch detailed match data."""
    url = f"{MATCH_URL}/lol/match/v5/matches/{match_id}"
    headers = {"X-Riot-Token": API_KEY}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    elif response.status_code == 429:  # Handle rate limiting
        retry_after = int(response.headers.get("Retry-After", 1))
        print(f"Rate limit hit. Retrying after {retry_after} seconds...")
        time.sleep(retry_after)
        return get_match_details(match_id)
    else:
        print(f"Error fetching match details: {response.status_code}, {response.text}")
        return None


In [34]:
def save_matches_to_csv(match_details_list, filename="matches.csv"):
    """Save match details to a CSV file."""
    matches_data = []
    for match in match_details_list:
        if match:  # Ensure the match is valid
            match_info = match["info"]
            for participant in match_info["participants"]:
                matches_data.append({
                    "match_id": match["metadata"]["matchId"],
                    "summoner_name": participant["summonerName"],
                    "champion": participant["championName"],
                    "role": participant["teamPosition"],
                    "win": participant["win"],
                    "kills": participant["kills"],
                    "deaths": participant["deaths"],
                    "assists": participant["assists"]
                })
    df = pd.DataFrame(matches_data)
    df.to_csv(filename, index=False)
    print(f"Saved {len(matches_data)} entries to {filename}")

In [ ]:
# --- Main Script ---
if __name__ == "__main__":
    summoner_name = input("Enter the summoner's name: ").strip()
    summoner_tagline = input("Enter the summoner's tagline: ").strip()
    
    # Step 1: Get Summoner Info
    summoner_info = get_summoner_info(summoner_name,summoner_tagline)
    if not summoner_info:
        print("Failed to fetch summoner info. Exiting.")
        exit(1)

    puuid = summoner_info["puuid"]
    print(f"Fetching match IDs for summoner: {summoner_name} (PUUID: {puuid})")

    # Step 2: Get Match IDs
    match_ids = get_match_ids(puuid, count=10)  # Adjust the count as needed
    print(f"Fetched {len(match_ids)} match IDs.")

    # Step 3: Get Match Details
    all_match_details = []
    for idx, match_id in enumerate(match_ids):
        print(f"Fetching details for match {idx + 1}/{len(match_ids)}: {match_id}")
        match_details = get_match_details(match_id)
        all_match_details.append(match_details)
        time.sleep(1.2)  # Avoid hitting rate limits

    # Step 4: Save to CSV
    save_matches_to_csv(all_match_details, filename="..\data\matches.csv")
    print("Data collection complete.")

Fetching match IDs for summoner: nay (PUUID: IvsWwkcWhgfifuaSvv9XoWw8OtzFDFvHj0lZpyN6yO-jSROwuQRiH2KczwahTZezGnRIGLUmfOJyug)
Fetched 10 match IDs.
Fetching details for match 1/10: EUW1_7189126563
Fetching details for match 2/10: EUW1_7186797526
Fetching details for match 3/10: EUW1_7184516358
Fetching details for match 4/10: EUW1_7183385240
Fetching details for match 5/10: EUW1_7182993484
Fetching details for match 6/10: EUW1_7182968265
Fetching details for match 7/10: EUW1_7182889300
Fetching details for match 8/10: EUW1_7182832101
Fetching details for match 9/10: EUW1_7182226910
Fetching details for match 10/10: EUW1_7182180649
Saved 100 entries to matches.csv
Data collection complete.


In [ ]:
# EOF